In [1]:
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

In [ ]:
pd.set_option(
    'display.max_rows', 100
)
url ="http://publicplansdata.org/api/"
focus_plans_list=['101', '110', '125', '179']